In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Parameters employed:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import math
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlxtend.regressor import StackingCVRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
# Loading training and testing data:
all_path = "/content/gdrive/My Drive/DM_Project_1/train.csv"
X_train_all = pd.read_csv(all_path)
test_path = "/content/gdrive/My Drive/DM_Project_1/test.csv"
test_data = pd.read_csv(test_path)
X_test_all = test_data.copy()

# Unwanted features:
uw_list = ["Id", "val_kf", "SalePrice"]

In [4]:
def feats_trfm(comp_features):
  new_feats = comp_features.copy()

  # Converting certain features to string type
  for cols in ['YrSold', 'MoSold', 'MSSubClass']:
    new_feats[cols] = new_feats[cols].astype(str)
  
  # Finding Numerical Columns and replacing NaN's with median:
  num_cols = [col for col in new_feats.columns if new_feats[col].dtype != 'object' and col not in uw_list]
  for cols in num_cols:
    new_feats[cols] = new_feats.groupby("Neighborhood")[cols].transform(lambda x: x.fillna(x.median()))

  # Finding Categorical Columns and replacing NaN's with "Missing":
  categ_cols = [col for col in new_feats.columns if new_feats[col].dtype == 'object']
  for cols in categ_cols:
    new_feats[cols] = new_feats[cols].fillna("Missing")
  
  # Simple feature engineering:
  new_feats['HouseSFOverall'] = new_feats['TotalBsmtSF']+new_feats['1stFlrSF']+new_feats['2ndFlrSF']
  new_feats['LotCompAr'] = new_feats['LotFrontage']+new_feats['LotArea']
  new_feats['BsmtFinComp'] = new_feats['BsmtFinSF1']+new_feats['BsmtFinSF2']
  new_feats['BathComp'] = new_feats['FullBath']+new_feats['HalfBath']
  new_feats['PorchComp'] = new_feats['OpenPorchSF']+new_feats['EnclosedPorch']+new_feats['ScreenPorch']

  # Label encoding required columns:
  reqd_cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 'ExterQual', 'ExterCond','HeatingQC', \
               'PoolQC', 'KitchenQual', 'BsmtFinType1', 'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', \
               'LandSlope', 'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'YrSold', 'MoSold')
  for col in reqd_cols:
    lbl = LabelEncoder() 
    lbl.fit(list(new_feats[col].values)) 
    new_feats[col] = lbl.transform(list(new_feats[col].values))

  # Replacing lower count features with string "Sparse"
  categ_cols = [col for col in new_feats.columns if new_feats[col].dtype == 'object']   
  for col in categ_cols:
    new_dict = new_feats[col].value_counts().to_dict()
    for key, value in new_dict.items():
      if value/len(new_feats[col]) < 0.02:
        new_feats.loc[:,col][new_feats[col] == key] = "Sparse"

  # Compute the skew values and based on the threshold apply the transformation:
  num_cols = [col for col in new_feats.columns if new_feats[col].dtype != 'object' and col not in uw_list]
  skew_thresh = new_feats[num_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
  skew_pd = pd.DataFrame({'Skew' :skew_thresh})
  skew_ind = skew_pd[abs(skew_pd) > 0.75].index
  for feat in skew_ind:
    new_feats[feat] = boxcox1p(new_feats[feat], 0.15)

  return new_feats

In [5]:
# Processing & Transformation of Train/Test features:
Test_Trfm_Feats = feats_trfm(X_test_all)
Test_Trfm_Feats.fillna(0, inplace=True) 

def train_model(fold, model, model_name, is_stack=False):

  # Defining 4 fold:
  kfolds = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
  X_train_all['val_kf'] = -1
  
  for idx, (Train_Ind, Val_Ind) in enumerate(kfolds.split(X_train_all, X_train_all['SalePrice'])):
    X_train_all.loc[Val_Ind, 'val_kf'] = idx
    
  Train_Trfm_Feats = feats_trfm(X_train_all)
  Train_Trfm_Feats['SalePrice'] = np.log1p(Train_Trfm_Feats['SalePrice'])

  # Concatenation of Train & Test Data, and apply Get_Dummies
  All_Trfm_Feats = pd.concat([Train_Trfm_Feats, Test_Trfm_Feats])
  All_Trfm_Feats = pd.get_dummies(All_Trfm_Feats)  
  Test_Data = All_Trfm_Feats.tail(1459)
  All_Data = All_Trfm_Feats.head(1460)
  reqd_cols = [col for col in All_Data.columns if col not in uw_list]    

  # Split Train & Validation Data:
  Train_Data = All_Data[All_Data['val_kf'] != fold].reset_index(drop=True)
  Train_Data = Train_Data[Train_Data['GrLivArea'] < 4500]
  Train_Data = Train_Data.reset_index(drop=True)
  Valid_Data = All_Data[All_Data['val_kf'] == fold].reset_index(drop=True)

  # Train stack and other models:
  if is_stack:
    model.fit(Train_Data[reqd_cols].values, Train_Data['SalePrice'])
  else:
    model.fit(Train_Data[reqd_cols].values, Train_Data['SalePrice'])
  
  print('{} Validation Root Mean Square Error : {}'.format(model_name, mean_squared_error(Valid_Data['SalePrice'], \
                                                                                          model.predict(Valid_Data[reqd_cols].values), squared=False)))
  return model.predict(Test_Data[reqd_cols].values)

In [6]:
print('Training LGBM, XGB, E-NET, SVR, GBR, RIDGE, LASSO & STACK GEN ...')

Test_Pred = pd.DataFrame()
for fold in range(3):
  model_xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bytree=0.7, gamma=0, importance_type='gain', objective='reg:linear', learning_rate=0.02, \
                         max_delta_step=0, max_depth=4, min_child_weight=1, n_estimators=1000, num_parallel_tree=1, random_state=42, reg_alpha=0.00007, \
                         reg_lambda=1, scale_pos_weight=1, subsample=0.8, tree_method='exact', validate_parameters=1)
  Test_Pred["xgboost" + str(fold) + "fld"] = train_model(fold, model_xgb, 'XGBoost') 

  model_ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=[14.5, 14.7, 14.9, 15, 15.1, 15.3, 15.5])) 
  Test_Pred["ridge" + str(fold) + "fld"] = train_model(fold, model_ridge, 'Ridge') 

  model_lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e+7, alphas=[0.00005, 0.0001, 0.0003, 0.0005, 0.0007, 0.0008], random_state=42))
  Test_Pred["lasso" + str(fold) + "fld"] = train_model(fold, model_lasso, 'Lasso') 

  model_enet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e+7, alphas=[0.0001, 0.0002, 0.0004, 0.0005, 0.0007], l1_ratio=[0.8, 0.9, 0.99, 1]))
  Test_Pred["elasticnet" + str(fold) + "fld"] = train_model(fold, model_enet, 'ElasticNet') 

  model_gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, \
                                      min_samples_split=10, loss='huber', random_state=42)
  Test_Pred["gradb" + str(fold) + "fld"] = train_model(fold, model_gbr, 'GradientBoost') 

  model_svr = make_pipeline(RobustScaler(), SVR(C=20, epsilon=0.008, gamma=0.0003))
  Test_Pred["svr" + str(fold) + "fld"] = train_model(fold, model_svr, 'SVR') 

  model_stack_gen = StackingCVRegressor(regressors=(model_ridge, model_lasso, model_enet, model_gbr, model_xgb, model_svr), meta_regressor=model_xgb, \
                                      use_features_in_secondary=True)
  Test_Pred["stack" + str(fold) + "fld"] = train_model(fold, model_stack_gen, 'StackGen', is_stack=True) 

print('Finished training all the models ...')

def weight_models_pred(Test_Pred):
  Y_test_pred = np.expm1(Test_Pred.iloc[:,0])
  Test_Pred = Test_Pred.drop(Test_Pred.columns[[0]], axis=1)
  for col in Test_Pred.columns:
    if col not in ['stack0fld', 'stack1fld', 'stack2fld']:
        Y_test_pred += np.expm1(Test_Pred[col])
    else:
        Y_test_pred += np.expm1(Test_Pred[col])*4
  return Y_test_pred/30

pandas_test = pd.DataFrame({'Id': test_data.Id, 'SalePrice': weight_models_pred(Test_Pred)})
pandas_test.to_csv('/content/gdrive/My Drive/DM_Project_1/submission.csv', index=False)

Training LGBM, XGB, E-NET, SVR, GBR, RIDGE, LASSO & STACK GEN ...
[15:57:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Validation Root Mean Square Error : 0.10713882828532732
Ridge Validation Root Mean Square Error : 0.13814481343624133
Lasso Validation Root Mean Square Error : 0.13548596248818706
ElasticNet Validation Root Mean Square Error : 0.1357232007247689
GradientBoost Validation Root Mean Square Error : 0.11894529626634916
SVR Validation Root Mean Square Error : 0.14172168472367247
[15:58:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:58:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:58:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:58:54] WARNING: /workspace/src/objecti